Let's first start spacy

In [165]:
# import required modules
import spacy
import pandas as pd

In [166]:
# Load english vocabulary
nlp = spacy.load("en_core_web_sm")

In [167]:
# check a small sentence
# check their tags
# check their NER tags

doc = nlp(u'Barrack Obama was the President of United States of America.')

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
print("")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Barrack barrack PROPN NNP compound Xxxxx True False
Obama obama PROPN NNP nsubj Xxxxx True False
was be VERB VBD ROOT xxx True True
the the DET DT det xxx True True
President president PROPN NNP attr Xxxxx True False
of of ADP IN prep xx True True
United united PROPN NNP compound Xxxxx True False
States states PROPN NNP pobj Xxxxx True False
of of ADP IN prep xx True True
America america PROPN NNP pobj Xxxxx True False
. . PUNCT . punct . False False

Barrack Obama 0 13 PERSON
United States of America 35 59 GPE


One way to fetch data from Wikipedia is to get the daily dumps provide by WikiMedia. But that is too large and parsing it for specific data according to our data may be a tedious task. however, let's see if we can parse similar data using one of the smaller datasets for wikibooks from https://dumps.wikimedia.org/backup-index.htmlLet's create a script to parse data from enwikibooks dump.

In [168]:
# Data has already been extracted from xml dump using wikiextractor to these files on my local machine.
# sample data is as show below
# there is a doc block containing the content
"""
<doc id="5" url="https://en.wikibooks.org/wiki?curid=5" title="Organic Chemistry/Cover">
Organic Chemistry/Cover

Welcome to the world's foremost open content<br>Organic Chemistry Textbook<br>on the web!

Organic chemistry is primarily devoted to the unique properties of the carbon atom and its compounds. These compounds play a critical role in biology and ecology, Earth sciences and
 geology, physics, industry, medicine and — of course — chemistry. At first glance, the new material that organic chemistry brings to the table may seem complicated and daunting, bu
t all it takes is concentration and perseverance. Millions of students before you have successfully passed this course and you can too!

This field of chemistry is based less on formulas and more on reactions between various molecules under different conditions. Whereas a typical general chemistry question may ask a
student to compute an answer with an equation from the chapter that they memorized, a more typical organic chemistry question is along the lines of "what product will form when subs
tance X is treated with solution Y and bombarded by light". The key to learning organic chemistry is to "understand" it rather than cram it in the night before a test. It is all wel
l and good to memorize the mechanism of Michael addition, but a superior accomplishment would be the ability to explain "why" such a reaction would take place.

As in all things, it is easier to build up a body of new knowledge on a foundation of solid prior knowledge. Students will be well served by much of the knowledge brought to this su
bject from the subject of General Chemistry. Concepts with particular importance to organic chemists are covalent bonding, Molecular Orbit theory, VSEPR Modeling, understanding acid
/base chemistry vis-a-vis pKa values, and even trends of the periodic table. This is by no means a comprehensive list of the knowledge you should have gained already in order to ful
ly understand the subject of organic chemistry, but it should give you some idea of the things you need to know to succeed in an organic chemistry test or course.

Organic Chemistry is one of the subjects which are very useful and close to our daily life. We always try to figure out some of the unknown mysteries of our daily life through our f
actious thinking habit, which generates superstitions. Through the help of chemistry we can help ourselves to get out of this kind of superstition.
We always try to find the ultimate truth through our own convenience. In the ancient past we had struggled to make things to go as per our need. In that context we have found fire,
house, food, transportation, etc…

Now the burning question is: "how can chemistry help our daily life?" To find the answer of this questions, we have to know the subject thoroughly. Let us start it from now.


</doc>
"""
fname = "/Users/saurabh/workspace/datasets/wiki/AA/wiki_00"

In [72]:
from nltk.tokenize import sent_tokenize, WordPunctTokenizer
import uuid

In [136]:
wtkz = WordPunctTokenizer()

In [74]:
"""
We assign each document with a unique page id.
We assign every line in the document with unique id.
"""
def process_file(fname):
    with open(fname) as f:
        lines = f.readlines()
    data = {}
    curr = False
    curr_data = []
    for line in lines:
        if line.startswith('</doc'):
            if len(curr_data) >= 1:
                data[doc_id] = curr_data
            curr = False
        if curr:
            if len(line) > 20:
                curr_data.append((doc_id, uuid.uuid4(), line.strip()))
        if line.startswith('<doc'):
            doc_id = line.split('<doc id="')[1].split('"')[0]
            curr = True
    return curr_data

In [75]:
data = process_file(fname)
print(data[0])
raw_df = pd.DataFrame(data, columns=["ids", "txt_id", "text"])

('5', UUID('51de6ee3-9c01-4290-9e16-73e8fb145918'), 'Organic Chemistry/Cover')


In [76]:
raw_df.head()

,ids,txt_id,text
0,5,51de6ee3-9c01-4290-9e16-73e8fb145918,Organic Chemistry/Cover
1,5,66ed753a-5612-40da-915c-e51948f658d1,Welcome to the world's foremost open content<b...
2,5,b5364df9-b4cf-4585-a1aa-2e80f9a112b1,Organic chemistry is primarily devoted to the ...
3,5,6e5e4e5b-a586-4829-b97c-71df9aef77b1,This field of chemistry is based less on formu...
4,5,f8966f48-11cc-46b5-a8a8-27fbcab2f62c,"As in all things, it is easier to build up a b..."


In [77]:
print(raw_df["ids"].count())

4542


In [78]:
# Apply spacy.nlp to all the documents. After this we will extract Name Entities dfrom the 
raw_df["nlp_txt"] = raw_df["text"].apply(nlp)

In [79]:
raw_df.head()

,ids,txt_id,text,nlp_txt
0,5,51de6ee3-9c01-4290-9e16-73e8fb145918,Organic Chemistry/Cover,"(Organic, Chemistry, /, Cover)"
1,5,66ed753a-5612-40da-915c-e51948f658d1,Welcome to the world's foremost open content<b...,"(Welcome, to, the, world, 's, foremost, open, ..."
2,5,b5364df9-b4cf-4585-a1aa-2e80f9a112b1,Organic chemistry is primarily devoted to the ...,"(Organic, chemistry, is, primarily, devoted, t..."
3,5,6e5e4e5b-a586-4829-b97c-71df9aef77b1,This field of chemistry is based less on formu...,"(This, field, of, chemistry, is, based, less, ..."
4,5,f8966f48-11cc-46b5-a8a8-27fbcab2f62c,"As in all things, it is easier to build up a b...","(As, in, all, things, ,, it, is, easier, to, b..."


In [169]:
# get named entities
def get_ners(doc):
    val = []
    for ent in doc.ents:
#         return ent.text, ent.start_char, ent.end_char, ent.label_
        val.append((ent.text, ent.label_))
    return val

In [81]:
data_new = raw_df["nlp_txt"].apply(get_ners)

In [91]:
data_new.loc[4]

[('General Chemistry', 'ORG'),
 ('Concepts', 'PERSON'),
 ('Molecular Orbit', 'PRODUCT'),
 ('Modeling', 'GPE')]

In [170]:
raw_df.loc[2, ["text"]][0]

'Organic chemistry is primarily devoted to the unique properties of the carbon atom and its compounds. These compounds play a critical role in biology and ecology, Earth sciences and geology, physics, industry, medicine and — of course — chemistry. At first glance, the new material that organic chemistry brings to the table may seem complicated and daunting, but all it takes is concentration and perseverance. Millions of students before you have successfully passed this course and you can too!'

We can run NERs on these dumps from wikipedia. However, let's try capturing our own custom data. We will now try to train NER. But before that we require annotated data. We will create data basis the following algorithm.

This script is a starter for Named Entity Recognition for Wikipedia data.
We have to do the following tasks in sequence:
1. Extract data from Wikipedia<br />
2. Annotate the data corresponding to an algorithm. Current algorithm employed<br />
    2.1 Start with seed_words. call `wikipedia.page(seed_word)`<br />
    2.2 For every page, take the title and the first line. Since, seed are musical words, if title is present in first line of the page. We mark it as MUSIC token.<br />
    2.3 Navigate to the first 5 links, most probably they are also music words. So, fetch first link, and again do 2.2.<br />
    2.4 Go to specified depth.<br />

In [92]:
import wikipedia

In [130]:
seed_words = ["musical instruments"]

In [131]:
def fetch_page_data(wiki_page, indent=0):
    data_map = {}
    print('\t'*indent ,wiki_page.title)
#     print('\t' ,wiki_page.content.split('.')[0])
#     print('\t' ,wiki_page.links[:4])
    data_map["title"] = wiki_page.title
    data_map["line1"] = wiki_page.content.split('.')[0]
    return data_map

In [132]:
def fetch_music_data(seed, data):
    if depth == 1:
        return
    search_pages = wikipedia.search(seed)
    print(sw)
    for sp in search_pages:
        wiki_page = wikipedia.page(sp)
        data.append(fetch_page_data(wiki_page))
        for wpl in wiki_page.links[:3]:
            lwpage = wikipedia.page(wpl)
            data.append(fetch_page_data(lwpage, 1))
    return data

In [133]:
data = []
for seed in seed_words:
    data = fetch_music_data(seed, data)

musical instruments
 Musical instrument
	 20th-century music
	 21st-century classical music
	 A-side and B-side
 List of musical instruments
	 20th-century music
	 21st-century classical music
	 A-side and B-side
 Fender Musical Instruments Corporation
	 Acoustic guitar
	 Adrian Belew
	 Amplifier
 Musical instrument classification
	 Accordion
	 Acoustic guitar
	 Adungu
 List of Indian musical instruments
	 Adana (raga)
	 Aerophone
	 Aerophone
 G&L Musical Instruments
	 Aerosmith
	 Alnico
	 Bass guitar
 Electronic musical instrument
	 ANS synthesizer
	 ARP Instruments
	 ARP Odyssey
 List of Chinese musical instruments
	 Alligator drum
	 List of Chinese musical instruments
	 Bamboo
 String instrument
	 3rd bridge
	 Accompaniment
	 Accordion
 Triangle (musical instrument)
	 Envelope (music)
	 Accordion
	 Acme siren


In [135]:
music_df = pd.DataFrame(data)
music_df.head()

,line1,title
0,A musical instrument is an instrument created ...,Musical instrument
1,During the 20th century there was a huge incre...,20th-century music
2,"21st-century classical music is art music, in ...",21st-century classical music
3,The terms A-side and B-side refer to the two s...,A-side and B-side
4,This is a list of musical instruments,List of musical instruments


In [148]:
def append_to_data(text, ret_data):
    tkns = wtkz.tokenize(text)
    for tkn in tkns:
        ret_data.append((tkn, 'O'))
    return ret_data

def split_and_mark(row):
    ret_data = []
    line1, title = row
    lline1 = line1.lower()
    ltitle = title.lower()
    idx = line1.lower().find(title.lower())
    if idx == -1:
        ret_data = append_to_data(line1, ret_data)
    else:
        end_idx = idx+len(title)
        sd = line1[:idx]
        ret_data = append_to_data(sd, ret_data)
        ret_data.append((line1[idx:end_idx], 'MUSIC'))
        sd = line1[end_idx:]
        ret_data = append_to_data(sd, ret_data)
    return ret_data

In [157]:
marked_data = music_df.apply(split_and_mark, axis=1)
marked_data.head(5)

0    [(A, O), (musical instrument, MUSIC), (is, O),...
1    [(During, O), (the, O), (20th, O), (century, O...
2    [(21st-century classical music, MUSIC), (is, O...
3    [(The, O), (terms, O), (A-side and B-side, MUS...
4    [(This, O), (is, O), (a, O), (list of musical ...
dtype: object

In [162]:
write_data = ''
for dlist in marked_data:
    for dtup in dlist:
        write_data += dtup[0] + '\t' + dtup[1] + '\n'
    write_data += '\n'

In [164]:
with open("output.txt", "w") as f:
    f.write(write_data)

In [173]:
# sample data from output.txt
"""
The     O
ANS synthesizer MUSIC
is      O
a       O
photoelectronic O
musical O
instrument      O
created O
by      O
Russian O
engineer        O
Evgeny  O
Murzin  O
from    O
1937    O
to      O
1957    O

ARP Instruments MUSIC
,       O
Inc     O

The     O
ARP Odyssey     MUSIC
is      O
an      O
analog  O
synthesizer     O
introduced      O
in      O
1972    O
"""

""

''